## 任務流程 ##

1. 前置作業 ＝> import 需要套件、建立相關身份認證資料
2. 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面（詳情：selenium-dcard應用），104為動態網頁，使 用selenium進行頁面滑動，依個人需求取得各職缺網址，並存成一個網址list
3. 透過迴圈進入各職缺網址內，取得相關資料，append進入output_list中，後續將此list轉成df
4. 將爬蟲回來的資料存成df，進行data cleaning，完成後輸出為excel檔！

## 前置作業 ＝> import 需要套件、建立headers&相關cookies資料
* 設置optins => 擋掉跳出視窗
* 建立headers
* 準備參數cookies (全職、關鍵字、清單條例式)

In [100]:
import time
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import openpyxl
import json
from pathlib import Path
base_path = Path(r'%notebook').absolute().parent.parent.parent
with open(fr"{base_path}\config.json", "r") as f:
    config = json.load(f)

In [101]:
# 參數設定
options = Options()
options.add_argument("--disable-notifications") # 禁用瀏覽器通知功能、彈出式的視窗
options.add_argument("--headless") # 啟用瀏覽器的無頭（headless）模式
options.chrome_executable_path = config['path']['chromedriver']

url_search = "https://www.104.com.tw/jobs/search/"
my_headers = {
    'User-Agent':config["request_headers"]["User-Agent"],
    'Accept':config["request_headers"]["Accept"],
    'Accept-Encoding':config["request_headers"]["Accept-Encoding"],
    'Accept-Language':config["request_headers"]["Accept-Language"],
    'Sec-Ch-Ua-Platform':config["request_headers"]["Sec-Ch-Ua-Platform"],
    'Sec-Fetch-Dest':config["request_headers"]["Sec-Fetch-Dest"],
    }
my_params = {'ro': '1',                # 限定全職的工作，如果不限定則輸入0
             'keyword': '數據分析',     # 想要查詢的關鍵字
             'mode': 'l'}              # 清單瀏覽模式

# 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面
* session連線 => 帶著my_params通過搜尋頁面，抵達搜尋結果頁面，104為動態網頁，使用selenium進行頁面滑動
* 輸入欲爬取至第幾頁面
* 建立url的beautiful soup物件，取得各職缺網址並存成work_url_lists
* 使用driver.page_source來取得所有頁面資訊
* 過濾掉置頂徵才廣告 => url中jobsource參數等於hotjob_chr者
* 取得url中的職缺編號，並組成app版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)

In [102]:
# session 連線
# 帶著參數通過搜尋頁面，抵達結果頁面的網址
ss = requests.session()
res = ss.get(url_search, headers=my_headers, params=my_params)

# 104為動態網頁，使用selenium進行頁面滑動，蒐集網址
# 輸入欲爬取至第幾頁面
page_number = 3
driver = Chrome(options=options)
driver.get(res.url)
for i in range(1, page_number+1):
    driver.execute_script("var s = document.documentElement.scrollTop=5000")
    print("目前爬至第{}頁".format(i))
    time.sleep(random.randint(4, 10))

# 建立beautiful soup物件
# 使用driver.page_source來取得所有頁面資訊
# 過濾掉置頂徵才廣告 => url中包含hotjob_chr參數
# 取得url中的職缺編號，並通過ajax的URL來找資料=>　https://www.104.com.tw/job/ajax/content/ + 職缺編號(ex. 5944x)
soup = BeautifulSoup(driver.page_source, features='html.parser')
url_data = [url["href"] for url in soup.select('li.job-mode__jobname a') if url["href"].split("=")[1] != 'hotjob_chr']
job_code_list = list(re.findall("job/(.*)\?", url)[0] for url in url_data)

# 關閉頁面
driver.close()

目前爬至第1頁
目前爬至第2頁
目前爬至第3頁


# 透過迴圈進入各職缺網址內，取得相關資料，append進入output中，後續將此轉成df
1. 每3秒爬一次，避免被鎖ip或是重導到電腦版網址，因而產生錯誤
2. 取得職缺相關資料
3. 更新時間、公司、職位、工作地點、工作內容
4. 工作需求技能、工具、其他條件
5. 額外資訊(聯絡人、薪水)
6. append進入output

In [104]:
# 每3秒爬一次，避免被鎖或是導向網頁，造成錯誤發生
output = []
num = 1
for job_code in job_code_list:
    # 每個頁面的url會因為job_code不一樣
    my_headers.update({"Referer": f"https://www.104.com.tw/job/{job_code}"})
    res_work = ss.get(f'https://www.104.com.tw/job/ajax/content/{job_code}', headers=my_headers, allow_redirects=False)
    w_soup = res_work.json()['data']
    
    try:
        # 更新時間、公司、職位、工作地點、工作內容
        update_time = w_soup['header']['appearDate']
        company = w_soup['header']['custName']
        title = w_soup['header']['jobName']
        work_place = w_soup['jobDetail']['addressRegion'] + w_soup['jobDetail']['addressDetail']
        description = w_soup['jobDetail']['jobDescription']
        
        # 工作需求技能、工具、其他條件
        job_skill = ','.join("'" + x + "'" for x in [item['description'] for item in w_soup['condition']['skill']])
        job_tool = ','.join("'" + x + "'" for x in [item['description'] for item in w_soup['condition']['specialty']])
        other = w_soup['condition']['other']

        ### 額外資訊(聯絡人、薪水)
        recruiter = w_soup['contact']['hrName']
        salary = w_soup['jobDetail']['salary']
        
    except ValueError as e:
        print("Value error", f"https://www.104.com.tw/job/{job_code}")
    except IndexError as e:
        print("Index error", f"https://www.104.com.tw/job/{job_code}")
        
    output.append([update_time, company, title, work_place, salary, description, job_skill, job_tool, other, recruiter, salary, f"https://www.104.com.tw/job/{job_code}"])
    
    print(f"第{num}筆資料跑完, 共{len(job_code_list)}筆資料。 URL: https://www.104.com.tw/job/{job_code}")
    num +=1
    time.sleep(3)

第1筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/85i5i
第2筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/7bu1t
第3筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/86se2
第4筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/842b7
第5筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/872uo
第6筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/5944x
第7筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/7e6go
第8筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/85hlz
第9筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/86b3m
第10筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/86moz
第11筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/86rjv
第12筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/85hmc
第13筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/85trd
第14筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/7na0r
第15筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/7xdj3
第16筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/7rboq
第17筆資料跑完, 共120筆資料。 URL: https://www.104.com.tw/job/7fp2e
第18筆資料跑完, 共120筆資料。 URL: https://www.104.

In [105]:
df = pd.DataFrame(output, 
                  columns=['update_time', 'company', 'title', 'work_place', 'salary', 
                           'description', 'job_skill', 'job_tool', 'other', 'recruiter',
                           'salary','url'])
df

,update_time,company,title,work_place,salary,description,job_skill,job_tool,other,recruiter,salary,url
0,2023/12/21,洛憶股份有限公司,數據分析師,台北市大安區敦化南路2段57號12樓,"月薪30,000~45,000元",1.蒐集追蹤相關產業數據，維持數據來源品質及穩定性 \n2.執行數據分析專案與任務，提供報表...,,"'Excel','Google Analytics'",,李小姐,"月薪30,000~45,000元",https://www.104.com.tw/job/85i5i
1,2023/12/19,宏景智權科技股份有限公司,數據分析師,新北市中和區中正路928號5樓,"月薪35,000~45,000元","1. 熟悉統計學知識，可正確理解使用數據。\n2. 對數據分析具有統整運用能力,具有撰寫良好...","'提案與簡報技巧','市場調查資料分析與報告撰寫'",,請提供統計相關資料或成績單供參考,林先生,"月薪35,000~45,000元",https://www.104.com.tw/job/7bu1t
2,2023/12/12,奇點數據行銷有限公司,數據分析師,台中市北屯區文心路四段696號10樓之2,待遇面議,職務說明\n1. 根據營運需求建構預測模型\n2. 執行需求及數據分析\n3. 提供資訊視覺...,,"'Python','Tableau'",其他條件\n* 請寫自傳。\n* 需具備 (Tableau / Power BI / Goo...,謝元峻,待遇面議,https://www.104.com.tw/job/86se2
3,2023/12/20,風采有限公司,數據分析師(擴編),台中市西屯區惠中路一段,"月薪45,000~65,000元",■工作內容\n 1.數據處理-收集並處理內部與外部數據，確保數據完整與正確性，創建清晰可用...,,"'Python','Tableau','Power BI​'",加分條件:\n1.具備Data Mining，Machine Learning概念尤佳\n2...,NakoHuang,"月薪45,000~65,000元",https://www.104.com.tw/job/842b7
4,2023/12/21,國泰健康管理顧問股份有限公司,【國泰集團】商業數據分析師 Business Data Analyst（Smart Health）,台北市大安區敦化南路二段333號7樓,待遇面議,我們的團隊致力於開創一個嶄新的Smart Health商業模式，\n我們的使命是透過「數據驅...,,"'Python','ETL','MySQL','AWS'",我們期待您擁有以下能力與潛力：\n1、對數據分析有基本的能力或理解，並有願意學習和成長的態度...,人力資源部郭小姐,待遇面議,https://www.104.com.tw/job/872uo
...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023/12/19,愛酷智能科技股份有限公司,【產品開發】行銷數據應用顧問/數據分析師,台北市松山區敦化南路一段2號5樓,待遇面議,愛酷智能科技致力於發展企業在行銷科技（MarTech）的應用，與跨渠道的對話式商務整合、藉由...,"'市場調查資料分析與報告撰寫','網站流量成效追蹤','資料庫軟體應用'",'MySQL',,郭小姐,待遇面議,https://www.104.com.tw/job/7uer4
116,2023/12/18,姿研國際股份有限公司,會員管理 數據分析師,台北市中山區南京東路二段100號6樓,待遇面議,1.會員資料收集、標籤定義與數據整理。\n2.進行調查數據的彙整，分析調查數據，並撰寫分析建...,"'行銷策略擬定','顧客關係管理','市場調查資料分析與報告撰寫'","'Tableau','Power BI​'",,吳小姐,待遇面議,https://www.104.com.tw/job/840jq
117,2023/12/19,91APP_九易宇軒股份有限公司,［MKT］資深CRM數據分析師/ 資深品牌數據分析顧問,台北市南港區八德路四段768巷5號6樓,"年薪650,000~1,300,000元",● 91APP 提供什麼樣的服務？\n91APP成立於2013年，為國內首家專精於虛實融合 ...,,"'MS SQL','Excel','PowerPoint'","1.數據解讀能力佳\n2.統計背景，良好邏輯能力\n3.熟悉資料庫(SQL)/具備R, Py...",HR Jessie,"年薪650,000~1,300,000元",https://www.104.com.tw/job/5t187
118,2023/12/20,玩構網路科技有限公司,【GenApe】AI產品企劃、數據分析、UI/UX優化（AI Product Planner）,高雄市楠梓區,"月薪35,000~55,000元",產品介紹 https://www.genape.ai/\n產品入口 https://app....,"'專案溝通╱整合管理','專案管理架構及專案說明','提案與簡報技巧','網站企劃能力','...","'HTML','CSS','Planner','Google Analytics','Goo...",1.若有作品集或作品網址請附上,蔡雅婷,"月薪35,000~55,000元",https://www.104.com.tw/job/7ga0s


In [106]:
# 去除重複值 => 透過company, title, work_place來篩選掉相同重複值缺，保留第一個(日期較新)
df2 = df.copy()
df2.drop_duplicates(subset=['company', 'title', 'work_place'], keep="first", inplace=True)
df2.sort_values(by="update_time", ascending=False, inplace=True)
df2

,update_time,company,title,work_place,salary,description,job_skill,job_tool,other,recruiter,salary,url
74,2023/12/22,弈樂科技股份有限公司,數據分析師(碩士),台中市西屯區安和東路3號6樓,"月薪44,000~69,000元",***應徵時請附上作品集、文案、自傳、證照等加分文件***\n1.\t需求分析及規劃資料萃取...,,"'C','C#','C++','Java','Matlab','Python','R','E...",1.\t有使用R、Tableau、PowerBi、Google Data Studio、Go...,HR,"月薪44,000~69,000元",https://www.104.com.tw/job/7n8pl
112,2023/12/22,群益金鼎證券股份有限公司,【資訊】大數據分析人員,台北市松山區民生東路三段156號13F,待遇面議,若您對大數據分析與運用有高度興趣，且具備金融顧客經營/證券數據分析能力，歡迎加入我們，成為金...,"'資料庫系統管理維護','資料庫程式設計','資料庫軟體應用'","'Java','ETL','MS SQL','MySQL'",1. 對數據分析與證券資料具高度熱情，具邏輯性、願意接受挑戰且有責任感。\n2. 數理統計相...,粘經理,待遇面議,https://www.104.com.tw/job/6la7u
108,2023/12/22,樺康智雲股份有限公司,智慧建築及能管數據分析師,台北市大安區和平東路一段180號7樓,"月薪40,000~70,000元",全球城市與建築面臨巨大的近零考驗，但智慧建築的智慧節能，並非透過人力密集的方式加以實現，我們...,,"'Python','MS SQL','PostgreSQL'",,蔡明達,"月薪40,000~70,000元",https://www.104.com.tw/job/86poa
32,2023/12/22,玉山銀行_玉山商業銀行股份有限公司,【個人金融】通路數據分析人員(經驗行員),台北市松山區,待遇面議,1. 營業單位績效分析與模型建立\n2. 設計績效管理相關儀表板與數據串接\n3. 參與各式...,,"'Python','Tableau'",邏輯能力佳，具理財、放款行銷經驗，熟悉SQL等資料庫程式語言，對業務數據分析、程式撰寫具熱忱,陳小姐,待遇面議,https://www.104.com.tw/job/80r3f
65,2023/12/22,弈樂科技股份有限公司,數據分析師(學士),台中市西屯區安和東路3號6樓,"月薪35,000~60,000元",***應徵時請附上作品集、文案、自傳、證照等加分文件***\n1.\t需求分析及規劃資料萃取...,,"'C','C#','C++','Java','Matlab','Python','R','E...",1.\t有使用R、Tableau、PowerBi、Google Data Studio、Go...,HR,"月薪35,000~60,000元",https://www.104.com.tw/job/6j51q
...,...,...,...,...,...,...,...,...,...,...,...,...
30,2023/12/04,昱勝資訊股份有限公司,數據分析師-學生實習/兼職,台北市文山區羅斯福路六段218號5樓,時薪180~200元,1、參與數據專案、協助開發數據模型，各領域之專案協助\n2、協助分析師維運NLP模型，進行數...,,,資料分析相關經驗(修習相關課程/培訓/專案/研究),王先生,時薪180~200元,https://www.104.com.tw/job/85j4e
40,2023/11/15,百阜科技股份有限公司,【擴編新團隊】數據分析師,台北市內湖區瑞湖街17號7樓,待遇面議,1.\t彙整遊戲數據報表並將資料視覺化，以協助相關人員了解產品狀況。\n2.\t建立分析指標...,"'市場調查資料分析與報告撰寫','統計軟體操作','調查樣本統計分析','資料庫軟體應用'",,●具備以下條件優先面試(非必要條件)\n\n1.\t熟悉統計理論相關知識並有實際數據分析經驗...,HR,待遇面議,https://www.104.com.tw/job/758eh
17,2023/11/02,立達國際電子股份有限公司,管理部（宜科）- 數據分析,宜蘭縣宜蘭市宜科南路15號3樓之8,"月薪35,000~75,000元",職位類別A - 電商運營數據分析師\n1. 對店商高成長有熱忱\n2. 對店商實務超作有耐心...,,'鼎新',,Andrea Shih,"月薪35,000~75,000元",https://www.104.com.tw/job/851th
39,2023/10/03,聯新國際醫院,數位管理處 數據分析師,桃園市平鎮區廣泰路77號,"月薪41,000~54,000元",1.整合各式資料來源與分析設計\n2.設計數據分析模型去達成策略需求\n3.數據分析工具評估...,,"'Matlab','Python','R','SAS'",1. 2年相關數據分析的實務經驗\n2. 有相關研究計畫或專案計畫作品尤佳\n3. 參與過K...,傅小姐,"月薪41,000~54,000元",https://www.104.com.tw/job/7u6uu


In [107]:
# 存成excel檔
df2.to_excel("./data/104_job_bank(data analysis related).xlsx", index=False)